In [ ]:
! pip install datasets openai
from datasets import load_dataset
dataset = load_dataset("truthful_qa","multiple_choice")

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(dataset['validation'], test_size=0.2, random_state=42)

In [ ]:
from openai import OpenAI
import openai
import re
from tqdm import tqdm
import os
import csv
start = 0
max = len(train_set['question'])
client = OpenAI() ### api_key = xxx
prefix = '''Read the multiple-choice question, analyze step by step, select the correct option and give option letter e.g. A or B as your answer. \
Use the following format to provide your answer and confidence level: \n
Explanation: [insert step-by-step analysis here]\n
Answer and Confidence (0-100): [Your answer, please only include the capital letter, e.g. B], \
[Your confidence level, please only include the numerical number, e.g. 80]% \n
Note: The confidence level indicates the degree of certainty you have about your answer and is represented as a percentage. \
For instance, if your confidence level is 80%, it means you are 80% certain that your answer is correct and there is a 20% chance that it may be incorrect.\n\n'''
for question,data in tqdm(zip(train_set['question'][start:max],train_set['mc1_targets'][start:max])):
    options = [f"{chr(65+i)}. {choice}" for i, choice in enumerate(data['choices'])]
    correct_answer = [chr(65+i) for i, label in enumerate(data['labels']) if label == 1]
    input = question + '\n' + '\n'.join(options)
    prompt = prefix + f"Question: [{input}]"
    response = client.chat.completions.create(
      model="gpt-4",
      messages=[
        {"role": "system", "content": "You are a chatbot"},
        {"role": "user", "content": prompt},
      ]
    )
    output = response.choices[0].message.content.replace('.','')
    print(f"{output}\n")
    explanation = re.search(r'Explanation: (.*)\n', output)
    explanation = explanation.group(1) if explanation else output
    answer_confidence = re.search(r'Answer and Confidence \(0-100\): (.+), (\d+)%', output)
    answer = answer_confidence.group(1).strip() if answer_confidence else output
    confidence_level = int(answer_confidence.group(2)) if answer_confidence else output

    # Write data to CSV
    with open('cot_trainset_gpt4.csv', 'a+', newline='') as file:
        writer = csv.writer(file)

        # Writing headers
        if os.path.getsize('cot_trainset_gpt4.csv') == 0:
            writer.writerow(["Question", "Options", "Predicted Answer", "Correct Answer", "Confidence Level", "Explanation"])

        # Writing data
        writer.writerow([question, "\n".join(options), answer, correct_answer[0], confidence_level, explanation])